# ⚠️ UPDATED: Calibrator Imports Simplified

**Migration Update:** We've simplified the calibrator architecture by removing the unnecessary `UnifiedVolatilityCalibrator` factory pattern. 

Now users directly choose the appropriate calibrator:
- **`LocalVolatilityCalibrator`**: For gradient-based optimization (SLSQP, L-BFGS-B) - fast local search
- **`GlobalVolatilityCalibrator`**: For global optimization (Differential Evolution, Multi-start) - robust global search

This is more explicit and eliminates unnecessary abstraction layers.

# Unified Volatility Model Calibration

This notebook demonstrates the **unified volatility calibrator framework** for comparing different volatility models using a single, consistent calibration approach.

## Key Features
- **UnifiedVolatilityCalibrator**: Single calibrator that works with any volatility model
- **Multiple Models**: Compare Time-Adjusted Wing Model vs Traditional Wing Model  
- **Visual Comparison**: Interactive plots showing model performance
- **Performance Metrics**: RMSE comparison and parameter analysis

## Architecture Benefits
- Mathematically consistent optimization across all models
- Easy extensibility for new volatility models
- Unified parameter bounds and constraint handling
- Backward compatibility with existing calibrators

## 1. Setup and Configuration

In [1]:
# Import required libraries
import sys
import os
import plotly.graph_objects as go
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Project setup
current_dir = os.getcwd()
project_root = os.path.dirname(current_dir)
sys.path.append(project_root)

print(f"📁 Current directory: {current_dir}")
print(f"📁 Project root: {project_root}")
print("✅ Environment setup complete")

📁 Current directory: /home/user/Python/Baseoffset-Fitting-Manager/notebooks
📁 Project root: /home/user/Python/Baseoffset-Fitting-Manager
✅ Environment setup complete


In [2]:
# Create configuration dictionary
config = {
    'date': '20240229',
    'option_expiry': '8MAR24',#'15MAR24',
    'snapshot_time': '2024-02-29T20:12:00',
    'snapshot_tolerance_seconds': 30    
}

print("📋 Configuration created:")
print(f"  - Expiry: {config['option_expiry']}")
print(f"  - Date: {config['date']}")
print(f"  - Snapshot time: {config['snapshot_time']}")

📋 Configuration created:
  - Expiry: 8MAR24
  - Date: 20240229
  - Snapshot time: 2024-02-29T20:12:00


## 2. Data Loading and Preparation

In [3]:
# Load market data
from utils.volatility_fitter.processed_data_loader import create_snapshot_option_chain, load_baseoffset_results, load_option_market_data
from utils.volatility_fitter.volatility_calculator import get_option_chains

date_str = config['date']
snapshot_time = datetime.strptime(config['snapshot_time'], "%Y-%m-%dT%H:%M:%S")
df_baseoffset = load_baseoffset_results(date_str)
df_option_md = load_option_market_data(date_str)

df_snapshot_md = create_snapshot_option_chain(df_option_md, df_baseoffset, snapshot_time)
my_expiry = config['option_expiry']

df_option_chain = get_option_chains(df_snapshot_md, my_expiry, snapshot_time)

print(f"📊 Loaded {len(df_option_chain)} option contracts")
print(f"📅 Expiry: {my_expiry} Time: {snapshot_time}")
df_option_chain.head()

Available expiries: ['27SEP24', '26APR24', '8MAR24', '3MAR24', '1MAR24', '29FEB24', '28JUN24', '2MAR24', '27DEC24', '29MAR24', '15MAR24', '22MAR24', '31MAY24']
📊 Loaded 41 option contracts
📅 Expiry: 8MAR24 Time: 2024-02-29 20:12:00


timestamp,bid_price,ask_price,strike,bid_size,ask_size,bid_price_P,ask_price_P,S,bid_price_fut,ask_price_fut,expiry,tau,bid_size_P,ask_size_P,future_basis
datetime[ns],f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64
2024-02-29 20:12:00,0.312,0.3385,42000,6.4,5.0,0.0003,0.0005,61924.86,62229.04,62229.04,"""8MAR24""",0.020525,7.5,7.9,304.18
2024-02-29 20:12:00,0.2805,0.307,44000,5.0,5.0,0.0005,0.0007,61924.86,62229.04,62229.04,"""8MAR24""",0.020525,0.2,18.2,304.18
2024-02-29 20:12:00,0.2645,0.291,45000,5.0,5.0,0.0005,0.0008,61924.86,62229.04,62229.04,"""8MAR24""",0.020525,3.4,17.4,304.18
2024-02-29 20:12:00,0.2485,0.2635,46000,5.0,6.3,0.0006,0.0009,61924.86,62229.04,62229.04,"""8MAR24""",0.020525,3.1,8.3,304.18
2024-02-29 20:12:00,0.2335,0.2585,47000,5.0,5.0,0.0008,0.0011,61924.86,62229.04,62229.04,"""8MAR24""",0.020525,4.0,12.2,304.18


In [4]:
df_snapshot_md[['expiry','r']].unique().to_dicts()

[{'expiry': '22MAR24', 'r': 0.2403},
 {'expiry': '31MAY24', 'r': 0.1739},
 {'expiry': '26APR24', 'r': 0.1763},
 {'expiry': '27DEC24', 'r': 0.1272},
 {'expiry': '1MAR24', 'r': 0.1856},
 {'expiry': '29MAR24', 'r': 0.1753},
 {'expiry': '27SEP24', 'r': 0.1391},
 {'expiry': '28JUN24', 'r': 0.1541},
 {'expiry': '2MAR24', 'r': 0.2094},
 {'expiry': '15MAR24', 'r': 0.2142},
 {'expiry': '8MAR24', 'r': 0.3109},
 {'expiry': '3MAR24', 'r': 0.2287}]

In [5]:
from utils.pricer.option_constraints import tighten_option_spread
from utils.reporting.html_table_generator import generate_price_comparison_table
from IPython.display import HTML, display

tightened_option_chain = tighten_option_spread(df_option_chain)
print(f"Comparison of old bid/ask proces and the tightened bid/ask price")

# Generate HTML table using external module
display(HTML(generate_price_comparison_table(tightened_option_chain, table_width="70%", font_size="10px")))

Comparison of old bid/ask proces and the tightened bid/ask price


In [6]:
tightened_option_chain

timestamp,bid_price,ask_price,strike,bid_size,ask_size,bid_price_P,ask_price_P,S,bid_price_fut,ask_price_fut,expiry,tau,bid_size_P,ask_size_P,future_basis,original_bid_price,original_ask_price,original_bid_price_P,original_ask_price_P,tightened_bid_price,tightened_ask_price,tightened_bid_price_P,tightened_ask_price_P
datetime[ns],f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2024-02-29 20:12:00,0.312,0.327892,42000,6.4,5.0,0.0003,0.0005,61924.86,62229.04,62229.04,"""8MAR24""",0.020525,7.5,7.9,304.18,0.312,0.3385,0.0003,0.0005,0.312,0.327892,0.0003,0.0005
2024-02-29 20:12:00,0.2805,0.295594,44000,5.0,5.0,0.0005,0.0007,61924.86,62229.04,62229.04,"""8MAR24""",0.020525,0.2,18.2,304.18,0.2805,0.307,0.0005,0.0007,0.2805,0.295594,0.0005,0.0007
2024-02-29 20:12:00,0.2645,0.279446,45000,5.0,5.0,0.0005,0.0008,61924.86,62229.04,62229.04,"""8MAR24""",0.020525,3.4,17.4,304.18,0.2645,0.291,0.0005,0.0008,0.2645,0.279446,0.0005,0.0008
2024-02-29 20:12:00,0.2485,0.263297,46000,5.0,6.3,0.0006,0.0009,61924.86,62229.04,62229.04,"""8MAR24""",0.020525,3.1,8.3,304.18,0.2485,0.2635,0.0006,0.0009,0.2485,0.263297,0.0006,0.0009
2024-02-29 20:12:00,0.2335,0.247149,47000,5.0,5.0,0.0008,0.0011,61924.86,62229.04,62229.04,"""8MAR24""",0.020525,4.0,12.2,304.18,0.2335,0.2585,0.0008,0.0011,0.2335,0.247149,0.0008,0.0011
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-02-29 20:12:00,0.012,0.0125,68000,15.4,1.4,0.103,0.108,61924.86,62229.04,62229.04,"""8MAR24""",0.020525,67.5,67.5,304.18,0.012,0.0125,0.103,0.108,0.012,0.0125,0.103,0.108
2024-02-29 20:12:00,0.0075,0.0085,70000,44.6,49.3,0.1305,0.136,61924.86,62229.04,62229.04,"""8MAR24""",0.020525,67.4,68.0,304.18,0.0075,0.0085,0.1305,0.136,0.0075,0.0085,0.1305,0.136
2024-02-29 20:12:00,0.005,0.0055,72000,0.5,19.8,0.16,0.166,61924.86,62229.04,62229.04,"""8MAR24""",0.020525,67.4,67.5,304.18,0.005,0.0055,0.16,0.166,0.005,0.0055,0.16,0.166


In [7]:
# Get forward price and time to expiry
from utils.volatility_fitter.volatility_calculator import process_option_chain_with_volatilities, process_volatility_with_greeks

df_option_with_vola = process_option_chain_with_volatilities(tightened_option_chain, interest_rate=0.30)
df_option_with_vola_and_greeks = process_volatility_with_greeks(df_option_with_vola)

assert len(df_option_with_vola['F'].unique().to_list()) == 1
assert len(df_option_with_vola['tau'].unique().to_list()) == 1
forward_price = df_option_with_vola['F'][0]
time_to_expiry = df_option_with_vola['tau'][0]

print(f"💰 Forward price: ${forward_price:,.2f}")
print(f"⏰ Time to expiry: {time_to_expiry:.4f} years")


print("✅ DataFrame with vega calculations created successfully using centralized module!")
df_option_with_vola_and_greeks.select(['strike', 'bidVola', 'midVola', 'askVola', 'vega'])

💰 Forward price: $62,229.04
⏰ Time to expiry: 0.0205 years
✅ DataFrame with vega calculations created successfully using centralized module!


strike,bidVola,midVola,askVola,vega
i64,f64,f64,f64,f64
42000,111.71,115.33,118.95,1.71
44000,106.27,108.76,111.24,2.5
45000,100.11,103.49,106.87,2.75
46000,96.43,99.34,102.26,3.18
47000,94.21,96.58,98.95,3.92
…,…,…,…,…
68000,72.9,73.5,74.1,25.88
70000,73.85,75.35,76.85,20.67
72000,76.44,77.41,78.38,15.99


In [8]:
fig = go.Figure()
# Plot with error bars for bid/ask implied volatility
fig.add_trace(go.Scatter(
    x=df_option_with_vola_and_greeks['strike'],
    y=(df_option_with_vola_and_greeks['bidVola_C']+df_option_with_vola_and_greeks['askVola_C'])/2,
    error_y=dict(type='data', array=(df_option_with_vola_and_greeks['askVola_C'] - df_option_with_vola_and_greeks['bidVola_C']).abs()/2,
        visible=True, color='blue'),
    mode='markers',
    name='Call IV (Bid/Ask Error Bar)',
    marker=dict(color='blue', symbol='circle'),
    opacity=0.8
))
fig.add_trace(go.Scatter(
    x=df_option_with_vola_and_greeks['strike'],
    y=(df_option_with_vola_and_greeks['bidVola_P']+df_option_with_vola_and_greeks['askVola_P'])/2,
    error_y=dict(type='data', array=(df_option_with_vola_and_greeks['askVola_P'] - df_option_with_vola_and_greeks['bidVola_P']).abs()/2,
        visible=True, color='orange'),
    mode='markers',
    name='Put IV (Bid/Ask Error Bar)',
    marker=dict(color='orange', symbol='circle'),
    opacity=0.8
))

fig.add_vline(x=df_option_with_vola_and_greeks['S'][0], line=dict(color='green', dash='dash'), name='Spot Price (S)')

fig.update_layout(
    title=f"{snapshot_time.strftime('%Y-%m-%d %H:%M')}: IV on DERIBIT BTC option for Expiry {my_expiry}",
    xaxis_title="Strike Price",
    yaxis_title="Implied Volatility (%)",
    legend_title="Legend",
    template="plotly_white"
)
fig.show()

## 3. Unified Volatility Model Calibration

Compare different volatility models using the unified calibrator framework:

In [9]:
# Import unified calibrator and both models
from utils.volatility_fitter.unified_volatility_calibrator import UnifiedVolatilityCalibrator
from utils.volatility_fitter.time_adjusted_wing_model.time_adjusted_wing_model import TimeAdjustedWingModel
from utils.volatility_fitter.wing_model.wing_model import WingModel
# 🔧 Load Volatility Configuration
from config.config_loader import load_volatility_config

# Extract required data from the processed dataframe
strikes_list = df_option_with_vola_and_greeks['strike'].to_list()
market_vols = (df_option_with_vola_and_greeks['midVola']/100).to_list()  # Convert from percentage
market_vegas = df_option_with_vola_and_greeks['vega'].to_list()

print("🚀 Unified Volatility Model Calibration")
print("=" * 50)
print(f"Market Data: {len(strikes_list)} strikes")
print(f"Forward Price: {forward_price:.2f}")
print(f"Time to Expiry: {time_to_expiry:.4f} years")
print(f"Strike range: ${min(strikes_list):,.0f} - ${max(strikes_list):,.0f}")

# Dictionary to store results for comparison
model_results = {}
vol_config = load_volatility_config()

🚀 Unified Volatility Model Calibration
Market Data: 41 strikes
Forward Price: 62229.04
Time to Expiry: 0.0205 years
Strike range: $42,000 - $76,000


### 3.1 Time-Adjusted Wing Model Calibration

In [10]:
# 1️⃣ Time-Adjusted Wing Model Calibration with Configuration Toggle
from utils.volatility_fitter.time_adjusted_wing_model import create_time_adjusted_wing_model_from_result
import numpy as np

print("🔧 Checking Time-Adjusted Wing Model Configuration...")

# Check if Time-Adjusted Wing Model is enabled in configuration
if vol_config.time_adjusted_wing_model_enabled:
    print("✅ Time-Adjusted Wing Model: ENABLED")
    print("🔧 Calibrating Time-Adjusted Wing Model...")

    initial_guess = [0.7, 0.01, 1.0, 1.0, -0.42, 0.2, 0.5, 0.5]  # Example initial guess

    # Create initial parameters using the helper function (creates WingModelParameters with proper mapping)
    ta_initial_params = create_time_adjusted_wing_model_from_result(
        result=initial_guess,
        forward_price=forward_price,
        ref_price=forward_price,  # Use same as forward price
        time_to_expiry=time_to_expiry
    )
    print(f"Initial parameters for Time-Adjusted Wing Model: {ta_initial_params}")
    # Get calibrator settings from configuration
    calibration_config = vol_config.get_calibration_config()
    unified_config = calibration_config.get('unified_calibrator', {})

    # Create unified calibrator using OPTIMIZED settings for large datasets
    ta_calibrator = UnifiedVolatilityCalibrator(
        model_class=TimeAdjustedWingModel,
        enable_bounds=unified_config.get('enable_bounds', True), 
        tolerance=1e-8,  # Relaxed tolerance for better convergence with 41 data points
        method=vol_config.calibration_method,
        max_iterations=2000,  # Increased iterations for complex optimization
        arbitrage_penalty=unified_config.get('arbitrage_penalty', 1e5)
    )

    print(f"🔧 OPTIMIZED Calibrator settings:")
    print(f"   Method: {vol_config.calibration_method}")
    print(f"   Tolerance: 1e-8 (optimized from {vol_config.calibration_tolerance})")
    print(f"   Max iterations: 2000 (optimized from {vol_config.max_calibration_iterations})")

    # Get parameter bounds from configuration using the corrected method
    ta_bounds = vol_config.get_parameter_bounds('time_adjusted_wing_model')

else:
    print("⚠️ Time-Adjusted Wing Model: DISABLED in configuration")
    print("   Skipping Time-Adjusted Wing Model calibration")
    ta_calibrator = None
    ta_initial_params = None
    ta_bounds = None

🔧 Checking Time-Adjusted Wing Model Configuration...
✅ Time-Adjusted Wing Model: ENABLED
🔧 Calibrating Time-Adjusted Wing Model...
Initial parameters for Time-Adjusted Wing Model: WingModelParameters(vr=0.7000, sr=0.0100, pc=1.0000, cc=1.0000, dc=0.2000, uc=-0.4200, dsm=0.5000, usm=0.5000)
🔧 OPTIMIZED Calibrator settings:
   Method: SLSQP
   Tolerance: 1e-8 (optimized from 1e-10)
   Max iterations: 2000 (optimized from 1000)


In [11]:
# Perform Time-Adjusted Wing Model calibration if enabled
if vol_config.time_adjusted_wing_model_enabled:
    # Perform calibration with optimized settings
    ta_result = ta_calibrator.calibrate(
        initial_params=ta_initial_params,
        strikes=strikes_list,  # Python lists for better compatibility
        market_volatilities=market_vols,
        market_vegas=market_vegas,
        parameter_bounds=ta_bounds,
        enforce_arbitrage_free=True
    )
    # Store results
    model_results['Time-Adjusted Wing'] = {'result': ta_result,'calibrator': ta_calibrator,'model_class': TimeAdjustedWingModel}

    if ta_result.success:
        print(f"✅ Time-Adjusted Wing Model calibration: SUCCESS")
        print(f"   Final Error: {ta_result.error:.6f}")
        
        # Check against configuration threshold
        if ta_result.error <= vol_config.max_rmse_threshold:
            print(f"   🎯 Error below config threshold ({vol_config.max_rmse_threshold:.3f}): ✅ PASS")
        else:
            print(f"   ⚠️ Error above config threshold ({vol_config.max_rmse_threshold:.3f}): REVIEW NEEDED")  
        # Show parameter values using the updated mapping
        print(f"   Optimized parameters: {ta_result.parameters}")
    else:
        print(f"❌ Time-Adjusted Wing Model calibration: FAILED")
        print(f"   Error: {ta_result.message}")
else:
    print("⚠️ Time-Adjusted Wing Model calibration: SKIPPED (disabled in configuration)")
    ta_result = None


✅ Time-Adjusted Wing Model calibration: SUCCESS
   Final Error: 0.147853
   ⚠️ Error above config threshold (0.100): REVIEW NEEDED
   Optimized parameters: WingModelParameters(vr=0.7014, sr=0.0207, pc=1.2405, cc=0.2756, dc=-0.0496, uc=0.3176, dsm=0.6205, usm=2.0000)


In [21]:
# 1️⃣ Time-Adjusted Wing Model Calibration with Configuration
from utils.volatility_fitter.wing_model import create_wing_model_from_result

# 2️⃣ Traditional Wing Model Calibration with Configuration Toggle
print("🔧 Checking Traditional Wing Model Configuration...")

# Check if Traditional Wing Model is enabled in configuration
if vol_config.wing_model_enabled:
    print("✅ Traditional Wing Model: ENABLED")
    print("🔧 Calibrating Traditional Wing Model...")

    initial_guess = [0.7025, 0.012, 1.05, 0.95, -0.22, 0.18, 0.48, 0.52]  # Example initial guess
    # Create initial parameters using configuration values
    wing_initial_params = create_wing_model_from_result(
        result=initial_guess,
        forward_price=forward_price,
        ref_price=forward_price,
        time_to_expiry=time_to_expiry
    )

    print(f"Initial parameters for Traditional Wing Model: {wing_initial_params}")

    # Get parameter bounds from configuration
    wing_bounds = vol_config.get_parameter_bounds('wing_model')

    # Create calibrator using OPTIMIZED settings for large datasets
    wing_calibrator = UnifiedVolatilityCalibrator(
        model_class=WingModel,
        enable_bounds=unified_config.get('enable_bounds', True),
        tolerance=1e-8,  # Relaxed tolerance for better convergence
        method=vol_config.calibration_method,
        arbitrage_penalty=unified_config.get('arbitrage_penalty', 1e5),
        max_iterations=2000  # Increased iterations for complex optimization
    )

    print(f"🔧 OPTIMIZED Wing calibrator settings:")
    print(f"   Method: {vol_config.calibration_method}")
    print(f"   Tolerance: 1e-8 (optimized from {vol_config.calibration_tolerance})")
    print(f"   Max iterations: 2000 (optimized from {vol_config.max_calibration_iterations})")

else:
    print("⚠️ Traditional Wing Model: DISABLED in configuration")
    wing_calibrator = None
    wing_initial_params = None
    wing_bounds = None


🔧 Checking Traditional Wing Model Configuration...
⚠️ Traditional Wing Model: DISABLED in configuration


In [22]:
# Calibrate Traditional Wing Model if enabled
if vol_config.wing_model_enabled:
    # Calibrate with optimized settings and Python lists
    wing_result = wing_calibrator.calibrate(
        initial_params=wing_initial_params,
        strikes=strikes_list,  # Python lists for better compatibility
        market_volatilities=market_vols,
        market_vegas=market_vegas,
        parameter_bounds=wing_bounds,
        enforce_arbitrage_free=True
    )

    # Store results
    model_results['Traditional Wing'] = {'result': wing_result,'calibrator': wing_calibrator,'model_class': WingModel}

    if wing_result.success:
        print(f"✅ Traditional Wing Model calibration: SUCCESS")
        print(f"   Final Error: {wing_result.error:.6f}")
        
        # Check against configuration threshold
        if wing_result.error <= vol_config.max_rmse_threshold:
            print(f"   🎯 Error below config threshold ({vol_config.max_rmse_threshold:.3f}): ✅ PASS")
        else:
            print(f"   ⚠️ Error above config threshold ({vol_config.max_rmse_threshold:.3f}): REVIEW NEEDED")    
        # Show parameter values using the updated mapping
        print(f"   Optimized parameters: {wing_result.parameters}")
    else:
        print(f"❌ Traditional Wing Model calibration: FAILED")
        print(f"   Error: {wing_result.message}")
else:
    print("⚠️ Traditional Wing Model calibration: SKIPPED (disabled in configuration)")
    wing_result = None

⚠️ Traditional Wing Model calibration: SKIPPED (disabled in configuration)


### 3.2 Model Comparison & Visualization

In [24]:
# 📊 Model Performance Comparison with Configuration Validation
print("\n📊 Model Performance Summary:")
print("=" * 50)

# Get validation config
validation_config = vol_config.get_validation_config()
quality_config = validation_config.get('quality', {})

print(f"🎯 Configuration thresholds:")
print(f"   Max RMSE: {vol_config.max_rmse_threshold:.3f}")
print(f"   Min R²: {quality_config.get('min_r_squared', 0.8):.3f}")

for model_name, data in model_results.items():
    result = data['result']
    rmse = result.error
    
    # Check against configuration thresholds
    rmse_pass = rmse <= vol_config.max_rmse_threshold
    status = "✅ PASS" if result.success and rmse_pass else "❌ FAIL"
    
    print(f"{model_name:25}: RMSE = {rmse:.6f} {status}")


📊 Model Performance Summary:
🎯 Configuration thresholds:
   Max RMSE: 0.100
   Min R²: 0.800
Time-Adjusted Wing       : RMSE = 0.147853 ❌ FAIL
Time-Adjusted Wing (Multi-Start): RMSE = 0.147853 ❌ FAIL


In [ ]:
# 📈 Generate Volatility Surface Comparison Plot
# Create extended strike range for smooth curves
min_strike = min(strikes_list) * 0.9
max_strike = max(strikes_list) * 1.1
extended_strikes = np.linspace(min_strike, max_strike, 100)

fig = go.Figure()

# Extract bid/ask volatilities for error bars
market_bid_vols = (df_option_with_vola_and_greeks['bidVola']/100).to_list()
market_ask_vols = (df_option_with_vola_and_greeks['askVola']/100).to_list()

# Add market data points with bid/ask error bars
fig.add_trace(go.Scatter(
    x=strikes_list, y=market_vols, mode='markers', name='Market IV (Bid/Ask)',
    error_y=dict(type='data', symmetric=False,
        array=[ask - mid for ask, mid in zip(market_ask_vols, market_vols)],  # Upper error
        arrayminus=[mid - bid for bid, mid in zip(market_bid_vols, market_vols)],  # Lower error
        visible=True, color='rgba(0,0,0,0.3)', thickness=2, width=3
    ),
    marker=dict(size=6, color='black', symbol='circle'),
    hovertemplate='Strike: %{x}<br>Mid Vol: %{y:.4f}<br>Bid-Ask Spread<extra></extra>',
    opacity=0.8
))
# Add vertical lines for reference
forward_price = df_option_with_vola_and_greeks['F'][0]    
fig.add_vline(x=forward_price, line=dict(color='purple', dash='dot', width=2), annotation_text=f"Forward: {forward_price:.0f}")

# Define colors for models
model_colors = ['blue', 'red', 'orange', 'purple', 'cyan', 'magenta', 'lime', 'pink']
color_map = {model_name: model_colors[i % len(model_colors)] for i, model_name in enumerate(model_results.keys())}

# Add model curves with fitted error bars
for model_name, data in model_results.items():        
    result = data['result']
    model_class = data['model_class']
    color = color_map.get(model_name, 'green')
    
    # Create model instance with optimized parameters
    model = model_class(result.parameters)
    
    # Calculate model volatilities for extended strikes
    model_vols = []
    for strike in extended_strikes:
        try:
            vol = model.calculate_volatility_from_strike(strike)
            model_vols.append(vol)
        except:
            model_vols.append(np.nan)
    
    # Add model curve
    fig.add_trace(go.Scatter(
        x=extended_strikes, y=model_vols, mode='lines',
        name=f'{model_name} (RMSE: {result.error:.6f})',
        line=dict(color=color, width=2),
        hovertemplate=f'{model_name}<br>Strike: %{{x}}<br>Vol: %{{y:.4f}}<extra></extra>'
    ))    

# Update layout
title = f"{(ts:=df_option_with_vola_and_greeks['timestamp'][0])}: Fitted Model vs Market Data - {my_expiry} Expiry"

fig.update_layout(xaxis_title='Strike Price', yaxis_title='Implied Volatility', width=900, height=600,
    title=dict(
        text=f"{title}<br><span style='font-size:14px'>Forward: {forward_price:.0f} | τ: {time_to_expiry:.4f}",# | RMSE: {fitter_result.error:.6f}<br> {param_string}</span>",
        x=0.5
    ),
    hovermode='closest', legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)'), template='plotly_white'
)

fig.show()

: 

### 3.3 Multi-Start Calibration to Escape Local Minima

The Time-Adjusted Wing Model got stuck in a poor local minimum. Let's use multiple starting points to find better global minima:

In [36]:
# 🎯 Multi-Start Calibration to Escape Local Minima (Configuration-Aware)
print("🎯 MULTI-START CALIBRATION FOR BETTER GLOBAL MINIMA")
print("=" * 60)

# Check if Time-Adjusted Wing Model is enabled and has results
if vol_config.time_adjusted_wing_model_enabled and 'ta_result' in locals() and ta_result is not None:
    try:
        # Use calibrate_with_multiple_starts method with correct signature
        ta_multistart_result = ta_calibrator.calibrate_with_multiple_starts(
            initial_params=ta_initial_params,  # Template for generating random starts
            strikes=strikes_list,
            market_volatilities=market_vols,
            market_vegas=market_vegas,
            parameter_bounds=ta_bounds,  # Used to generate random starts within bounds
            num_starts=10,  # Try 10 different starting points
            enforce_arbitrage_free=True
        )
        
        print(f"\n✅ Multi-Start Calibration Results:")
        print(f"   Success: {ta_multistart_result.success}")
        print(f"   Best Error: {ta_multistart_result.error:.6f}")
        print(f"   Number of attempts: 10")
        
        # Compare with single-start result
        original_rmse = ta_result.error
        if ta_multistart_result.error > 0 and original_rmse > 0:
            improvement_factor = original_rmse / ta_multistart_result.error
            improvement_percent = (original_rmse - ta_multistart_result.error) / original_rmse * 100
        else:
            improvement_factor = float('inf') if ta_multistart_result.error == 0 else 0
            improvement_percent = 100 if ta_multistart_result.error < original_rmse else 0
        
        print(f"\n📈 Improvement Analysis:")
        print(f"   Original Error: {original_rmse:.6f}")
        print(f"   Multi-start Error: {ta_multistart_result.error:.6f}")
        if improvement_factor != float('inf'):
            print(f"   Improvement factor: {improvement_factor:.2f}x")
        print(f"   Improvement: {improvement_percent:.2f}%")
        
        if ta_multistart_result.error <= vol_config.max_rmse_threshold:
            print(f"   🎯 Multi-start error below config threshold ({vol_config.max_rmse_threshold:.3f}): ✅ PASS")
        else:
            print(f"   ⚠️ Multi-start error above config threshold ({vol_config.max_rmse_threshold:.3f}): REVIEW NEEDED")
        
        # Show optimized parameters
        if ta_multistart_result.success:
            print(f"\n🎯 Original Parameters:              {ta_result.parameters}")  
            print(f"\n🎯 Optimized Multi-Start Parameters: {ta_multistart_result.parameters}")            
            
            # Update model results with the better result
            model_results['Time-Adjusted Wing (Multi-Start)'] = {
                'result': ta_multistart_result,
                'calibrator': ta_calibrator,
                'model_class': TimeAdjustedWingModel
            }
            
            print(f"\n🔄 Updated model_results with improved Time-Adjusted Wing Model!")
            
            # Check if this is significantly better
            if ta_multistart_result.error < original_rmse * 0.5:  # At least 50% improvement
                print(f"🎉 SIGNIFICANT IMPROVEMENT! Multi-start found much better solution!")
            elif ta_multistart_result.error < original_rmse:
                print(f"✅ IMPROVEMENT! Multi-start found better solution!")
            else:
                print(f"ℹ️ Multi-start result similar to original (local minimum issue may persist)")
        
    except Exception as e:
        print(f"\n🚨 Multi-start calibration failed: {e}")
        import traceback
        traceback.print_exc()

🎯 MULTI-START CALIBRATION FOR BETTER GLOBAL MINIMA

✅ Multi-Start Calibration Results:
   Success: True
   Best Error: 0.147853
   Number of attempts: 10

📈 Improvement Analysis:
   Original Error: 0.147853
   Multi-start Error: 0.147853
   Improvement factor: 1.00x
   Improvement: 0.00%
   ⚠️ Multi-start error above config threshold (0.100): REVIEW NEEDED

🎯 Original Parameters:              WingModelParameters(vr=0.7014, sr=0.0207, pc=1.2405, cc=0.2756, dc=-0.0496, uc=0.3176, dsm=0.6205, usm=2.0000)

🎯 Optimized Multi-Start Parameters: WingModelParameters(vr=0.7014, sr=0.0207, pc=1.2405, cc=0.2756, dc=-0.0496, uc=0.3176, dsm=0.6205, usm=2.0000)

🔄 Updated model_results with improved Time-Adjusted Wing Model!
ℹ️ Multi-start result similar to original (local minimum issue may persist)

✅ Multi-Start Calibration Results:
   Success: True
   Best Error: 0.147853
   Number of attempts: 10

📈 Improvement Analysis:
   Original Error: 0.147853
   Multi-start Error: 0.147853
   Improvement fa

In [37]:
# Test Differential Evolution for Time-Adjusted Wing Model
print("Testing Differential Evolution to address local minima issues")

# Create new calibrator instance with the updated method
ta_de_calibrator = UnifiedVolatilityCalibrator(
    model_class=TimeAdjustedWingModel,
    enable_bounds=True,
    tolerance=1e-8,
    method="SLSQP",
    max_iterations=2000
)

# Verify the method exists
print(f"✅ Differential Evolution method available: {hasattr(ta_de_calibrator, 'calibrate_with_differential_evolution')}")
print(f"   Available methods: {[m for m in dir(ta_de_calibrator) if 'calibrate' in m and not m.startswith('_')]}")
print()

# Get quality threshold from configuration
error_threshold = vol_config.max_rmse_threshold

# Use multi-start result if available, otherwise use original result
if 'ta_multistart_result' in globals() and ta_multistart_result.success:
    ta_enhanced_result = ta_multistart_result
else:
    ta_enhanced_result = ta_result

if hasattr(ta_de_calibrator, 'calibrate_with_differential_evolution'):
    # Test Differential Evolution
    print("🚀 Running Differential Evolution...")
    print("   Population size: 15 x num_parameters")
    print("   Max iterations: 1000")
    print("   Method: Global optimization with local refinement")
    print()

    ta_de_result = ta_de_calibrator.calibrate_with_differential_evolution(
        initial_params=ta_initial_params,
        strikes=strikes_list,
        market_volatilities=market_vols,
        market_vegas=market_vegas,
        parameter_bounds=ta_bounds,
        enforce_arbitrage_free=True,
        popsize=15,  # Population size multiplier
        maxiter=1000,
        seed=42  # For reproducibility
    )

    print(f"✅ Differential Evolution Results:")
    print(f"   Success: {ta_de_result.success}")
    print(f"   Error: {ta_de_result.error:.6f}")
    print(f"   Message: {ta_de_result.message}")

    if ta_de_result.success:
        de_error = ta_de_result.error
        original_error = ta_result.error
        enhanced_error = ta_enhanced_result.error
        
        print(f"\n📈 Performance Comparison:")
        print(f"   Original Error:     {original_error:.6f}")
        print(f"   Multi-start Error:  {enhanced_error:.6f}")
        print(f"   DE Error:           {de_error:.6f}")
        
        # Calculate improvements
        de_vs_original = ((original_error - de_error) / original_error) * 100
        de_vs_multi = ((enhanced_error - de_error) / enhanced_error) * 100
        
        print(f"\n🎯 Improvement Analysis:")
        print(f"   DE vs Original:    {de_vs_original:+.2f}%")
        print(f"   DE vs Multi-start: {de_vs_multi:+.2f}%")
        
        # Quality assessment
        if de_error <= error_threshold:
            print(f"   Status: ✅ PASS (Error {de_error:.6f} below {error_threshold} threshold)")
        else:
            print(f"   Status: ⚠️ REVIEW (Error {de_error:.6f} above {error_threshold} threshold)")
        
        print(f"\n🎛️ Differential Evolution Parameters: {ta_de_result.parameters}")
        print(f"\n🎛️ Original Parameters:               {ta_result.parameters}")
        print(f"\n🎛️ Multi-start Parameters:            {ta_enhanced_result.parameters}")    
        # Update model results with the better result
        model_results['Time-Adjusted Wing (Differential-Evolution)'] = {
            'result': ta_de_result,
            'calibrator': ta_de_calibrator,
            'model_class': TimeAdjustedWingModel
        }    
        
    else:
        print(f"   ❌ Differential Evolution failed: {ta_de_result.message}")
        
else:
    print("❌ Differential Evolution method not found. Module may not have been updated correctly.")


Testing Differential Evolution to address local minima issues
✅ Differential Evolution method available: True
   Available methods: ['calibrate', 'calibrate_with_differential_evolution', 'calibrate_with_multiple_starts']

🚀 Running Differential Evolution...
   Population size: 15 x num_parameters
   Max iterations: 1000
   Method: Global optimization with local refinement

✅ Differential Evolution Results:
   Success: True
   Error: 0.064900
   Message: Differential Evolution successful (nfev: 12378)

📈 Performance Comparison:
   Original Error:     0.147853
   Multi-start Error:  0.147853
   DE Error:           0.064900

🎯 Improvement Analysis:
   DE vs Original:    +56.10%
   DE vs Multi-start: +56.10%
   Status: ✅ PASS (Error 0.064900 below 0.1 threshold)

🎛️ Differential Evolution Parameters: WingModelParameters(vr=0.7014, sr=0.0414, pc=0.2217, cc=0.1817, dc=-1.1528, uc=0.4558, dsm=1.7726, usm=1.3519)

🎛️ Original Parameters:               WingModelParameters(vr=0.7014, sr=0.0207, 